In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models, datasets
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image

import numpy as np
import pandas
import os
import shutil
import logging
import time

In [ ]:
base_path = './movielens1m'

In [ ]:
items_extra = os.path.join(base_path, 'items.csv')
posters_path = os.path.join(base_path,  'posters', 'data')
kg_id_map_path = os.path.join(base_path, 'item_id_2_kg_id.csv')

In [ ]:
kg_id_map_df = pandas.read_csv(kg_id_map_path)
movielensid_to_kg_id = kg_id_map_df.set_index('item_id').to_dict()['kg_id']

# Autoencoder - Images

In [ ]:
img_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


In [ ]:
class SDCAE(nn.Module):
    def __init__(self):
        super(SDCAE, self).__init__()
        #  Input size: [batch, 3, 224, 224]
        #  Output size: [batch, 3, 224, 224]
        self.conv1 = nn.Conv2d(3, 28, 4, stride=4, padding=0)           
        self.conv2 = nn.Conv2d(28, 56, 4, stride=4, padding=0)          
        self.conv3 = nn.Conv2d(56, 112, 4, stride=2, padding=0)

        self.linear1 = nn.Linear(4032, 32)
        self.linear1_t = nn.Linear(32, 4032)

        self.conv3_t = nn.ConvTranspose2d(112, 56, 4, stride=2, padding=0)
        self.conv2_t = nn.ConvTranspose2d(56, 28, 4, stride=4, padding=0)
        self.conv1_t = nn.ConvTranspose2d(28, 3, 4, stride=4, padding=0) 

    def encode(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.linear1(x))
        return x

    def decode(self, x):
        x = F.relu(self.linear1_t(x))
        x = x.reshape((-1, 112, 6, 6))
        x = F.relu(self.conv3_t(x))
        x = F.relu(self.conv2_t(x))
        x = F.relu(self.conv1_t(x))
        return x
  

    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded)
        return encoded, decoded

# Autoencoder - Text

In [ ]:
class SDTAE(nn.Module):
    def __init__(self, input_size):
        super(SDTAE, self).__init__()
        #  Input size: [batch, 12234]
        #  Output size: [batch, 12234]
        self.linear1 = nn.Linear(input_size, 512)           
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 32) 

        self.linear_t3 = nn.Linear(32, 256)           
        self.linear_t2 = nn.Linear(256, 512)           
        self.linear_t1 = nn.Linear(512, input_size)


    def encode(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        return x

    def decode(self, x):
        x = F.relu(self.linear_t3(x))
        x = F.relu(self.linear_t2(x))
        x = F.relu(self.linear_t1(x))
        return x
  
    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded)
        return encoded, decoded

# KG

## KG-Dataset

In [ ]:
kg_train_csv = os.path.join(base_path, 'kg_train.dat')
kg_test_csv = os.path.join(base_path, 'kg_test.dat')
kg_valid_csv = os.path.join(base_path, 'kg_valid.dat')

In [ ]:
import random


class KG_dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, kg_csv, entity_total):
        """
        Args:
            kg_csv (string): Dataframe file contaning KG triplets of (head tail relation).
        """
        self.entity_total = entity_total
        self.kg_data = pandas.read_csv(kg_csv, 
                                   sep='\t',
                                   header=None,names=['head', 'tail', 'relation'])
    def __len__(self):
        return len(self.kg_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        head, tail, relation = self.kg_data.iloc[idx]
        pos_triple = (head, tail, relation)
        head = int(head)
        tail = int(tail)
        relation = int(relation)

        # Get negative sample for the current priple
        neg_list = None
        if random.random() < 0.5:
            neg_list = self.corrupt_head_filter(pos_triple, self.entity_total) 
        else:
            neg_list = self.corrupt_tail_filter(pos_triple, self.entity_total)

        neg_head, neg_tail, neg_relation = neg_list
        neg_head = int(neg_head)
        neg_tail = int(neg_tail)
        neg_relation = int(neg_relation)


        sample = {'pos_head': head, 'pos_tail': tail, 'pos_relation': relation,
                 'neg_head': neg_head, 'neg_tail': neg_tail, 'neg_relation': neg_relation}

        return sample
    


    # Change the head of a triple randomly,
    def corrupt_head_filter(self, triple, entityTotal):
        newHead = random.randrange(entityTotal)
        return (newHead, triple[1],triple[2])

    # Change the tail of a triple randomly,
    def corrupt_tail_filter(self, triple, entityTotal, tailDicts=None):
        newTail = random.randrange(entityTotal)
        return (triple[0], newTail,triple[2])


In [ ]:
# Get entities_count and relations_count

kg_train_data = pandas.read_csv(kg_train_csv, sep='\t', header=None,names=['head', 'tail', 'relation'])
kg_test_data = pandas.read_csv(kg_test_csv, sep='\t', header=None,names=['head', 'tail', 'relation'])
kg_valid_data = pandas.read_csv(kg_valid_csv, sep='\t', header=None,names=['head', 'tail', 'relation'])

entities_count = pandas.concat([kg_train_data["head"],
                            kg_test_data["head"],
                            kg_valid_data["head"],
                           kg_train_data["tail"],
                           kg_test_data["tail"],
                           kg_valid_data["tail"],],axis=0).unique().max()

relations_count = pandas.concat([kg_train_data["relation"],
                            kg_test_data["relation"],
                            kg_valid_data["relation"]]
                            ,axis=0).unique().max()

entities_count, relations_count

In [ ]:
kg_train_dataset = KG_dataset(kg_train_csv, entities_count)
kg_test_dataset = KG_dataset(kg_test_csv, entities_count)
kg_valid_dataset = KG_dataset(kg_valid_csv, entities_count)

In [ ]:
kg_train_loader = torch.utils.data.DataLoader(kg_train_dataset,
                                             batch_size=512, shuffle=True,
                                             num_workers=0)

kg_valid_loader = torch.utils.data.DataLoader(kg_test_dataset,
                                             batch_size=512, shuffle=True,
                                             num_workers=0)

kg_test_loader = torch.utils.data.DataLoader(kg_valid_dataset,
                                             batch_size=512, shuffle=True,
                                             num_workers=0)

## KG - Embedding

In [ ]:
import torch.autograd as autograd


class KG_net(nn.Module):
    def __init__(self,
                L1_flag,
                embedding_size,
                entity_total,
                relation_total,
                ):
        super(KG_net, self).__init__()
        self.L1_flag = L1_flag
        self.embedding_size = embedding_size
        self.ent_total = entity_total + 1
        self.rel_total = relation_total + 1
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda:0" if use_cuda else "cpu")


        # init user and item embeddings
#         , padding_idx=self.ent_total-1
        self.ent_embeddings = nn.Embedding(self.ent_total, self.embedding_size)

        self.rel_embeddings = nn.Embedding(self.rel_total, self.embedding_size)
        self.proj_embeddings = nn.Embedding(self.rel_total, self.embedding_size * self.embedding_size)

        self.ent_embeddings = self.ent_embeddings.to(self.device)
        self.rel_embeddings = self.rel_embeddings.to(self.device)
        self.proj_embeddings = self.proj_embeddings.to(self.device)

        
  
    def forward(self, x):
        h, t, r = x

        h = h.to(self.device)
        t = t.to(self.device)
        r = r.to(self.device)
        
        h_e = self.ent_embeddings(h)
        t_e = self.ent_embeddings(t)
        r_e = self.rel_embeddings(r)
        proj_e = self.proj_embeddings(r)


        proj_h_e = self.projection_transR_pytorch(h_e, proj_e)
        proj_t_e = self.projection_transR_pytorch(t_e, proj_e)

        if self.L1_flag:
            score = torch.sum(torch.abs(proj_h_e + r_e - proj_t_e), 1)
        else:
            score = torch.sum((proj_h_e + r_e - proj_t_e) ** 2, 1)
        
        return score


    def projection_transR_pytorch(self, original, proj_matrix):
        ent_embedding_size = original.shape[1]
        rel_embedding_size = proj_matrix.shape[1] // ent_embedding_size
        original = original.view(-1, ent_embedding_size, 1)
        proj_matrix = proj_matrix.view(-1, rel_embedding_size, ent_embedding_size)
        return torch.matmul(proj_matrix, original).view(-1, rel_embedding_size)



USE_CUDA = torch.cuda.is_available()
def to_gpu(var):
    if USE_CUDA:
        return var.cuda()
    return var

class marginLoss(nn.Module):
    def __init__(self):
        super(marginLoss, self).__init__()

    def forward(self, pos, neg, margin):
        zero_tensor = to_gpu(torch.FloatTensor(pos.size()))
        zero_tensor.zero_()
        zero_tensor = autograd.Variable(zero_tensor)
        return torch.sum(torch.max(pos - neg + margin, zero_tensor))
    
    
def normLoss(embeddings, dim=1):
    norm = torch.sum(embeddings ** 2, dim=dim, keepdim=True)
    return torch.sum(torch.max(norm - to_gpu(autograd.Variable(torch.FloatTensor([1.0]))), to_gpu(autograd.Variable(torch.FloatTensor([0.0])))))


# Movielens1m loader

In [ ]:
class Movielens1m_train_dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, rating_data, user_item_dict, items_extra_csv, img_dir,
                 img_size, entities_count, img_transform=None,
                 text_trasform=None):
        """
        Args:
            rating_csv (string): Dataframe file contaning user item ratings.
            item_extrac_csv (string)L Path to csv file containing all the items 
                extra information
            img_dir (string): Directory with all the images.
            img_transform (callable, optional): Optional transform to be applied
                on a sample.
            text_trasform (callable, optional): Optional text preprocess to be
             applied for the movie summary. Deffalt is CountVectorizer
        """
        self.rating_data = rating_data
        self.user_item_dict = user_item_dict
        self.items_extras = pandas.read_csv(items_extra_csv)
        self.text_list = list(self.items_extras['text_summery'])
        self.text_transform = (CountVectorizer().fit(self.text_list).transform
                               if text_trasform is None else text_trasform)
        self.img_list = list(self.items_extras['img_name'])
        self.img_size = img_size
        self.img_dir = img_dir
        self.img_transform = img_transform

        self.unkwon_entity_idx = entities_count

    def __len__(self):
        return len(self.rating_data)

    def __get_item_extra__(self, item_idx):
        img_name = self.img_list[item_idx]
        if img_name not in ['<fialed>', '<failed>']:
            img_path = os.path.join(self.img_dir, img_name)
            image = Image.open(img_path)
            image = image.convert('RGB')
            if self.img_transform is not None:
                image = self.img_transform(image)
        else:
            image = torch.zeros(*self.img_size)

        text = self.text_list[item_idx]
        text = torch.from_numpy(self.text_transform([text]).todense()).float()

        if int(item_idx) in movielensid_to_kg_id.keys():
            entity = int(movielensid_to_kg_id[item_idx])
        else:
            entity = self.unkwon_entity_idx

        return image, text.view(-1), entity
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        user, item, rating = self.rating_data.iloc[idx]
        image, text, entity = self.__get_item_extra__(item)
        
        user = int(user)
        item = int(item)

        sample = {'user': user, 'item': item, 'rating': rating,
                  'image': image, 'text': text,'entity': entity}

        return sample


In [ ]:
train_csv = os.path.join(base_path, 'train.dat')
valid_csv = os.path.join(base_path, 'valid.dat')

In [ ]:
train_dataframe = pandas.read_csv(train_csv, sep='\t',
                                   names=['user', 'item', 'rating'],
                                   index_col=False)
valid_dataframe = pandas.read_csv(valid_csv, sep='\t',
                                   names=['user', 'item', 'rating'],
                                   index_col=False)

test_dataframe = pandas.read_csv(valid_csv, sep='\t',
                                   names=['user', 'item', 'rating'],
                                   index_col=False)

rating_mean = train_dataframe['rating'].mean()

In [ ]:
train_dataset = Movielens1m_train_dataset(rating_data=train_dataframe,
                                    user_item_dict=None,
                                    items_extra_csv=items_extra,
                                    img_dir=posters_path,
                                    img_size=(3, 224, 224),
                                    entities_count=entities_count, img_transform=img_transform)

valid_dataset = Movielens1m_train_dataset(rating_data=valid_dataframe,
                                    user_item_dict=None,
                                    items_extra_csv=items_extra,
                                    img_dir=posters_path,
                                    img_size=(3, 224, 224),
                                    entities_count=entities_count, img_transform=img_transform)

test_dataset = Movielens1m_train_dataset(rating_data=test_dataframe,
                                    user_item_dict=None,
                                    items_extra_csv=items_extra,
                                    img_dir=posters_path,
                                    img_size=(3, 224, 224),
                                    entities_count=entities_count, img_transform=img_transform)

In [ ]:
batch_size = 32

cke_train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=4)

cke_valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=4)

cke_test_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=4)

# CKE Model

In [ ]:
class BuildCKE(nn.Module):
    def __init__(self, n_users, n_items, img_net, text_net, kg_net, rating_mean, embedding_size=32):
        super(BuildCKE, self).__init__()
        self.img_ae = img_net
        self.text_ae = text_net
        self.kg_net = kg_net
        self.user_embedding = nn.Embedding(num_embeddings=n_users,
                                           embedding_dim=embedding_size)
        self.item_embedding = nn.Embedding(num_embeddings=n_items,
                                           embedding_dim=embedding_size)
        
        self.user_bias = nn.Embedding(num_embeddings=n_users,
                                           embedding_dim=1)
        self.item_bias = nn.Embedding(num_embeddings=n_items,
                                           embedding_dim=1)
        self.rating_mean = rating_mean 
        
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    def forward_train(self, inputs):
        batch_size = inputs['entity'].size(0)
        img_noise = noise_lambda*torch.randn(batch_size, 3, 224, 224).cuda()
        text_noise = noise_lambda*torch.randn(batch_size, 12234).cuda()

        kg_embedding = self.kg_net.ent_embeddings(inputs['entity'].cuda())
        text_embeddding, text_recon = self.text_ae(inputs['text'].cuda()+text_noise)
        img_embedding, img_recon = self.img_ae(inputs['image'].cuda()+img_noise)

        item_id_embedding = self.item_embedding(inputs['item'].cuda())
        
        user_bias = self.user_bias(inputs['user'].cuda())
        item_bias = self.item_bias(inputs['item'].cuda())


        item_embedding = (text_embeddding+kg_embedding+
                                  img_embedding+item_id_embedding)
        user_embedding = self.user_embedding(inputs['user'].cuda())

        preds = torch.sum(item_embedding*user_embedding, axis=1) +user_bias.squeeze(1) + item_bias.squeeze(1) + self.rating_mean

        return_vals = {'preds': preds, 'img_recon': img_recon,
                     'text_recon': text_recon, 'img_embeding': img_embedding,
                     'text_embedding': text_embeddding, 'user_embedding': user_embedding,
                     'item_embedding': item_id_embedding,
                     'kg_embedding': kg_embedding}
        return return_vals
    
    def forward(self, x):
        kg_embedding = self.kg_net.ent_embeddings(x['entity'].cuda())
        text_embeddding, _ = self.text_ae(x['text'].cuda())
        img_embedding, _ = self.img_ae(x['image'].cuda())

        item_id_embedding = self.item_embedding(x['item'].cuda())


        item_embedding = (text_embeddding+kg_embedding+
                                  img_embedding+item_id_embedding)
    
        user_embedding = self.user_embedding(x['user'].cuda())
        
        user_bias = self.user_bias(x['user'].cuda())
        item_bias = self.item_bias(x['item'].cuda())


        preds = torch.sum(item_embedding*user_embedding, axis=-1)+user_bias.squeeze(1) + item_bias.squeeze(1) + self.rating_mean

        return preds
    

def tensor_norm(tensor):
    return (tensor**2).sum()


def eval_cke(loader, device):
    rating_loss = nn.MSELoss()
    cke.module.eval()
    with torch.no_grad():
        mse_test_loss = 0.0
        for cke_test_inputs in loader:
            cke_test_preds = cke(cke_test_inputs)
            mse_loss = rating_loss(cke_test_preds, cke_test_inputs['rating'].type(torch.FloatTensor).to(device))
            mse_test_loss += mse_loss
        mse_test_loss = mse_test_loss/len(loader)
        return mse_test_loss

## Create Model

In [ ]:
img_net = SDCAE()
kg_net = KG_net(True, embedding_size=32, entity_total=entities_count,
               relation_total=relations_count)
text_net = SDTAE(input_size=12234)
cke = BuildCKE(n_users=6040, n_items=3952, img_net=img_net, text_net=text_net,
                kg_net= kg_net,  rating_mean=rating_mean, embedding_size=32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
cke = cke.to(device)
cke = nn.DataParallel(cke)

# Training

## KG model conf

In [ ]:
from torch.autograd import Variable as V
import torch.optim as optim

# model params
relation_total = relations_count + 1
embedding_size = 32

# optimizer
kg_learning_rate = 0.001

kg_optimizer = optim.Adagrad(cke.module.kg_net.parameters(), lr=kg_learning_rate,
                             weight_decay=1e-5)

# loss
margin = 1.0
KG_LAMBDA = 1.0
margin_loss = marginLoss()

## CKE model conf

In [ ]:
noise_lambda = 0.001
text_loss = nn.MSELoss()
img_loss = nn.MSELoss()
rating_loss = nn.MSELoss()
l2_lambda = 1e-1
cke_learning_rate = 0.001
cke_optimizer = optim.Adam(cke.parameters(),lr=cke_learning_rate)

##  Traning Loop

In [ ]:
logging.basicConfig(filename='./CKE_traning_mse_bias.log' ,level=logging.INFO)

In [ ]:
cke.train()
for epoch in range(2, 1000):
    cke_training_running_loss = 0.0
    rating_training_loss = 0.0
    kg_training_running_loss = 0.0
    start = time.time()
    for inputs_cke, inputs_kg in zip(cke_train_loader, kg_train_loader):
#         pos_inputs_cke, neg_inputs_cke = inputs_cke
        pos_inputs_cke = inputs_cke

        inputs_kg = inputs_kg

      # =============Preprocessing===========
        pos_head, pos_tail, pos_relation = inputs_kg["pos_head"], inputs_kg["pos_tail"], inputs_kg["pos_relation"]
        neg_head, neg_tail, neg_relation = inputs_kg["neg_head"], inputs_kg["neg_tail"], inputs_kg["neg_relation"]
        

        # ============CKE============
        cke_optimizer.zero_grad()
        
        pos_cke_outputs =  cke.module.forward_train(pos_inputs_cke)
  
        pos_img_recon_loss = img_loss(pos_cke_outputs['img_recon'].to(device), pos_inputs_cke['image'].to(device))
        pos_text_recon_loss = text_loss(pos_cke_outputs['text_recon'].to(device), pos_inputs_cke['text'].to(device))


        
        regularization_loss = (tensor_norm(pos_cke_outputs['user_embedding']) + 
                               tensor_norm(pos_cke_outputs['img_embeding'])+
                               tensor_norm(pos_cke_outputs['kg_embedding'])+
                               tensor_norm(pos_cke_outputs['text_embedding'])+
                               tensor_norm(pos_cke_outputs['item_embedding']))
        
        rating_mse = rating_loss(pos_cke_outputs['preds'], pos_inputs_cke['rating'].type(torch.FloatTensor).to(device))

        
        cke_loss = rating_mse + pos_img_recon_loss+ pos_text_recon_loss+ l2_lambda*regularization_loss
        cke_training_running_loss += cke_loss
        rating_training_loss += rating_mse
        
        cke_loss.backward()
        cke_optimizer.step()

        # ============KG============
        kg_outputs_poss = cke.module.kg_net((pos_head, pos_tail, pos_relation))
        kg_outputs_neg = cke.module.kg_net((neg_head, neg_tail, neg_relation))

        kg_margin = margin_loss(kg_outputs_poss, kg_outputs_neg, margin)
        ent_embeddings = cke.module.kg_net.ent_embeddings(torch.cat([pos_head, pos_tail, neg_head, neg_tail]).to(device))
        rel_embeddings = cke.module.kg_net.rel_embeddings(torch.cat([pos_relation, neg_relation]).to(device))

        kg_loss = kg_margin + normLoss(ent_embeddings) + normLoss(rel_embeddings)
        kg_training_running_loss += kg_loss
        kg_loss.backward()
        kg_optimizer.step()
        kg_optimizer.zero_grad()

    kg_training_running_loss = kg_training_running_loss/len(cke_train_loader)
    cke_training_running_loss = cke_training_running_loss/len(cke_train_loader)
    rating_training_loss = rating_training_loss/len(cke_train_loader)

    if epoch % 5 == 0:
        mse_test_loss = eval_cke(cke_valid_loader, device)
        
        end_time =  time.time()-start
        print(f'Epoch {epoch} | Time {end_time:2f} | CKE Loss {cke_training_running_loss:2f} | KG Loss {kg_training_running_loss:2f} | Train mse loss {rating_training_loss:2f} | Test mse loss {mse_test_loss:4f}')
        logging.info(f'Epoch {epoch} | Time {end_time:2f} | CKE Loss {cke_training_running_loss:2f} | KG Loss {kg_training_running_loss:2f} | Train mse loss {rating_training_loss:2f} | Test mse loss {mse_test_loss:4f}')
    else:
        end_time =  time.time()-start
        print(f'Epoch {epoch} | Time {end_time:2f} | CKE Loss {cke_training_running_loss:2f} | KG Loss {kg_training_running_loss:2f} | Train mse loss {rating_training_loss:4f}')
        logging.info(f'Epoch {epoch} | Time {end_time:2f} | CKE Loss {cke_training_running_loss:2f} | KG Loss {kg_training_running_loss:2f} | Train mse loss {rating_training_loss:4f}')
   
    if epoch % 10 == 0:
        state = cke.module.state_dict()
        torch.save(state, f'./models/cke_mse_bias_{epoch}_001.ckp.pth')
        
state = cke.module.state_dict()
torch.save(state, './models/cke_mse_bias_1000_001.ckp.pth')

# Test CKE

In [ ]:
state = torch.load('./models/cke_mse_bias_580_001.ckp.pth')
cke.module.load_state_dict(state)

## MSE Evaluation

In [ ]:
eval_cke(cke_test_loader, device)

## Recall and MAP evaluation

In [ ]:
test_data = pandas.read_csv(os.path.join(base_path, 'test_for_recall.csv'))

In [ ]:
test_dataset = Movielens1m_train_dataset(rating_data=test_data,
                                    user_item_dict=None,
                                    items_extra_csv=items_extra,
                                    img_dir=posters_path,
                                    img_size=(3, 224, 224),
                                    entities_count=entities_count, img_transform=img_transform)
cke_test_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=256, shuffle=False,
                                              num_workers=1)

In [ ]:
cke.module.eval()
with torch.no_grad():
    preds = []
    mae_test_loss = 0.0
    for batch_idx, cke_test_inputs in enumerate(cke_test_loader):
        cke_test_preds = cke(cke_test_inputs)
        preds.append(cke_test_preds)
        if batch_idx % 100 == 0:
            print(f'Finised {batch_idx} test batches')
    preds = torch.cat(preds)

In [ ]:
complete_csv = pandas.DataFrame({'user': test_data['user'],
                                 'item': test_data['item'],   
                                 'True_val': test_data['True_val'],
                                  'Preds': preds.detach().cpu().numpy()})

In [ ]:
complete_csv.to_csv(os.path.join(base_path, 'Cke_final.csv'), index=False)

### The @k evaluation itself is coded into the calc recall@k and MAP@k_v1.1 notebook